# Try  gensim

In [14]:
from gensim import corpora
from gensim import models
from collections import defaultdict

In [1]:
raw_corpus = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",              
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [2]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))

In [5]:
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in raw_corpus]

# Count word frequencies
#from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]
processed_corpus

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [7]:
dictionary = corpora.Dictionary(processed_corpus)
print(dictionary)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


In [8]:
print(dictionary.token2id)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}


In [13]:
new_doc = "Human computer interaction"
new_vec = dictionary.doc2bow(new_doc.lower().split())
new_vec

[(0, 1), (1, 1)]

In [10]:
bow_corpus = [dictionary.doc2bow(text) for text in processed_corpus]
bow_corpus

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]

In [16]:
#from gensim import models
# train the model
tfidf = models.TfidfModel(bow_corpus)
# transform the "system minors" string
tfidf[dictionary.doc2bow("system minors".lower().split())]

[(5, 0.5898341626740045), (11, 0.8075244024440723)]

In [22]:
tfidf[dictionary.doc2bow("time graph".lower().split())]

[(6, 0.8075244024440723), (10, 0.5898341626740045)]

## Gensim Doc2Vec

In [7]:
import gensim
import os
import collections
import smart_open
import random
import pickle

In [19]:
def save_obj(file, obj):
    with open(file, 'wb') as f:
        pickle.dump(obj, f)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [20]:
path = r'C:\Users\sociocom\Desktop\Crawler\backup\31May-4June 2018'

In [21]:
df_spl = load_obj(path + r"\translated_spl_tweet")
df_bkk = load_obj(path + r"\translated_bkk_tweet")
df_ham = load_obj(path + r"\translated_ham_tweet")

In [22]:
df_ham.head()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,855881532145360896,focus_regional,1002062196292182016,Police Düren - driver flung out of the car aft...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de
1,855881532145360896,focus_regional,1002062196334096384,Police Neustadt bei Coburg - Without license o...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de
2,855881532145360896,focus_regional,1002062196317270017,Police Gädheim - gelding with enigmatic neck i...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de
3,855881532145360896,focus_regional,1002062196304695297,Polizeipräsidium Südosthessen - Offenbach - Pr...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de
4,855881532145360896,focus_regional,1002062196296355840,Police Neustadt bei Coburg - Accident Exodus h...,2018-05-31 05:40:19,-1,-1,Germany,"5.866357, 47.270127","15.041854, 55.056823",-1,-1,-1,de


In [23]:
len(df_bkk)

37095

In [24]:
df_spl.head()

,acc_id,acc,tweet_id,text,created_time,ex_lat,ex_long,place,lot_sw,lot_ne,tweet_reply_id,tweet_reply_user_id,tweet_reply_username,lang
0,880139242558676993,gu3ds_,1002070123061051392,I just think it's funny how the debauch hj ta ...,2018-05-31 06:11:49,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt
1,71562250,Alexrodrigues73,1002070143873245184,@IvanCarlus @eaglesaopaulo kkkkkkkkk.... cai m...,2018-05-31 06:11:54,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",1002023363378851841,47545111,IvanCarlus,pt
2,883893293465186304,ester_querinoo,1002070146721222657,"Mother, everything that comes from you to me i...",2018-05-31 06:11:55,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt
3,4794804172,elizaberh_costa,1002070147404828672,General sleeping,2018-05-31 06:11:55,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt
4,33572317,thelukeset,1002070200508932096,people this series 13 reasons why eh mto compl...,2018-05-31 06:12:08,-1,-1,Sao Paulo,"-46.826039, -24.008814","-46.365052, -23.356792",-1,-1,-1,pt


In [26]:
df_spl["text"][4]

'people this series 13 reasons why eh mto complicated ne still good that there are some warnings'

In [29]:
df_bkk["text"][13]

'Horrible !! Sathorn Taksin Express Boat or Boat'